In [ ]:
!git clone https://github.com/nianlonggu/MemSum.git

In [ ]:
!pip install -q -r MemSum/requirements.txt

In [ ]:
from MemSum.src.summarizer import MemSum
from tqdm import tqdm
from rouge_score import rouge_scorer
import json
import numpy as np
import os

In [ ]:
FILENAME_MODEL="model_batch_5600.pt"
PATH_TO_MODEL="model/memsum-custom-data/"
FILENAME_VOCAB="vocabulary_200dim.pkl"
PATH_TO_VOCAB="model/word_embedding/"

In [ ]:
memsum_custom_data = MemSum( os.path.join(PATH_TO_MODEL,FILENAME_MODEL) , 
                  os.path.join(PATH_TO_VOCAB,FILENAME_VOCAB), 
                  gpu = 0 ,  max_doc_len = 500  )

In [ ]:
def extract_summary(document_of_list_shape):
    list_of_results=[]
    extracted_summary = memsum_custom_data.extract( [ document_of_list_shape ], 
                                       p_stop_thres = 0.6, 
                                       max_extracted_sentences_per_document = 50
                                      )[0]
    for element in document_of_list_shape:
        if element in extracted_summary:
            list_of_results.append(element)
    return list_of_results

In [ ]:
DOCUMENT_TEST=["Accord collectif d'activité partielle ", 'de longue durée (APLD)', 'Entre les soussignés :', '', 'La Société PRESTOMATIC SAS', 'Au capital de 150 000,00 euros', 'Immatriculée au registre du commerce et des sociétés ', 'sous le numéro 339 853 178 RCS STRASBOURG', 'Dont le siège social est situé 7 rue des Hirondelles 67110 GUNDERSHOFFEN ', 'Représentée par, agissant en qualité de Président,', '', 'Dénommée ci-dessous “La Société”,', "D'une part,", 'Et,', '', 'Le comité social et économique ayant voté à la majorité des membres titulaires présents, dont le procès-verbal est annexé au présent accord, ', '', "D'autre part,", 'Il a été conclu le présent accord.', 'ARTICLE 1 - Préambule', "Du fait de la crise sanitaire liée à l'épidémie de Covid-19, la société, qui compte à ce jour 33 salariés, est confrontée à une importante baisse d’activité, qui a déjà conduit la société à recourir à l’activité partielle classique, permettant de réduire la durée du travail tout en maintenant le niveau de salaire et en préservant l’emploi et les compétences des salariés.", "Compte tenu de ces mesures et du contexte sanitaire, économique et social actuel très contraint, un diagnostic sur la situation économique de la société et ses perspectives d'activité a été établi. "]

In [ ]:
extract_summary(DOCUMENT_TEST)

# Inference on Dataset

In [ ]:
!pip install -q pyarrow pandas

In [ ]:
import pandas as pd
import os

In [ ]:
PATH_TO_PARQUET="../../"
NOM_PARQUET_LABEL="Dares_accords_docx_sommaire_label_230811.parquet"
NOM_PARQUET_DATASET="Dares_1Kaccords_dataset_txt_label.parquet"

In [ ]:
df_label=pd.read_parquet(os.path.join(PATH_TO_PARQUET,NOM_PARQUET_LABEL))

In [ ]:
df_dataset=pd.read_parquet(os.path.join(PATH_TO_PARQUET,NOM_PARQUET_DATASET))
df_dataset=df_dataset.drop(columns=["label_txt"])

In [ ]:
def trim_new_line_on_start(string):
    if string.startswith("\n") or string.startswith(" ")or string.startswith("\t"):
        return trim_new_line_on_start(string[1:])
    else:
        return string

In [ ]:
df_join_ftext_sum=df_dataset.join(df_label)
df_join_ftext_sum["txt_trim"]=df_join_ftext_sum["txt"].apply(trim_new_line_on_start)
df_join_ftext_sum["txt_trim_list"]=df_join_ftext_sum[["txt_trim"]].apply(lambda x : x.str.split("\n"))

In [ ]:
df_join_ftext_sum["extracted_summary"]=df_join_ftext_sum["txt_trim_list"].apply(extract_summary)

In [ ]:
df_join_ftext_sum["sommaire_label_list"]=df_join_ftext_sum[["sommaire_label"]].apply(lambda x : x.str.split("\n"))

In [ ]:
df_extracted=df_join_ftext_sum[["sommaire_label_list","extracted_summary"]]

In [ ]:
df_extracted.to_parquet("Dares_accords_docx_sommaire_memsum_230826.parquet")